# Warning: This is not a good idea to have tests in a notebook!

# Test board_to_array

In [ ]:
import pandas as pd
import chess
import preprocess
import numpy as np

In [2]:
df = pd.read_csv("data/cleaned/chess_train_val_full.csv", nrows=100_000)
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   canonical_fen  100000 non-null  object 
 1   advantage      100000 non-null  float64
dtypes: float64(1), object(1)
memory usage: 10.3 MB


## Any position

In [3]:
fen_any = str(df.iloc[10, 0])
fen_any_mirrored = chess.Board(fen_any).mirror().fen()

print(fen_any)
print(fen_any_mirrored)

arr_any = preprocess.board_to_array(fen_any)
arr_any_mirrored = preprocess.board_to_array(fen_any_mirrored)

np.array_equal(arr_any, arr_any_mirrored)

n2q3r/4b1k1/3p1p2/2p3p1/8/6BN/PP2Q1PP/3R2K1 w - -
3r2k1/pp2q1pp/6bn/8/2P3P1/3P1P2/4B1K1/N2Q3R b - - 0 1


True

## Castling position

In [4]:
fen_castling = "4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1"
# fen_castling = str(
#    df[(df["canonical_fen"].str.split().str[2].isin(("-", "KQkq")) == False)].iloc[0, 0]
# )
fen_castling_mirrored = chess.Board(fen_castling).mirror().fen()
print(fen_castling)
print(fen_castling_mirrored)

arr_castling = preprocess.board_to_array(fen_castling)
arr_castling_mirrored = preprocess.board_to_array(fen_castling_mirrored)

np.array_equal(arr_castling, arr_castling_mirrored)

4k2r/6r1/8/8/8/8/3R4/R3K3 w Qk - 0 1
r3k3/3r4/8/8/8/8/6R1/4K2R b Kq - 0 1


True

## En passant

In [5]:
fen_ep = str(df[(df["canonical_fen"].str.split().str[3] != "-")].iloc[0, 0])

fen_ep_mirrored = chess.Board(fen_ep).mirror().fen()
print(fen_ep)
print(fen_ep_mirrored)

arr_ep = preprocess.board_to_array(fen_ep)
arr_ep_mirrored = preprocess.board_to_array(fen_ep_mirrored)

np.array_equal(arr_ep, arr_ep_mirrored)

1k1rq2r/pbp1bpp1/1pn1p2p/3pP3/2B2Pn1/1PN2N2/PBPPQ1PP/1KR4R w - d6
1kr4r/pbppq1pp/1pn2n2/2b2pN1/3Pp3/1PN1P2P/PBP1BPP1/1K1RQ2R b - d3 0 1


True

# Test Battles

In [ ]:
Board = chess.Board()

In [ ]:
import chess
import preprocess
import torch
import numpy as np
from collections import Counter
import torch
import torch.nn as nn
from torch.distributions import Categorical
import random


@torch.inference_mode()
def choose_best_move(board: chess.Board, model: nn.Module, epsilon):
    model.eval()

    # collect numpy-arrays for each candidate move
    arrays = []
    moves = list(board.legal_moves)

    # forced move
    if len(moves) == 1:
        return moves[0]

    # if epsilon is or <= 0.0
    if not epsilon or epsilon <= 0.0:
        return random.choice(moves)

    for move in moves:
        board.push(move)
        fen = board.fen()
        arr = preprocess.board_to_array(fen)
        arrays.append(arr)
        board.pop()

    batch_np = np.stack(arrays, axis=0)

    batch = torch.from_numpy(batch_np).to(dtype=torch.float32)

    # their advantage
    preds = model(batch)
    preds = preds.squeeze()

    # best move -> least advantage for them
    return moves[int(torch.argmin(preds))]


def fight(model1: nn.Module, model2: nn.Module, epsilon, max_randomness):
    board = chess.Board()

    outcome = board.outcome()

    turn = 0
    while outcome is None:
        turn += 1
        if max_randomness and turn > max_randomness * 2:
            epsilon = None

        if board.turn == chess.WHITE:
            move = choose_best_move(board, model1, epsilon)
        else:
            move = choose_best_move(board, model2, epsilon)
        board.push(move)
        board.apply_mirror()

        outcome = board.outcome()

    return outcome, board.fen()


def war(model1: nn.Module, model2: nn.Module, games_per_side=10, temperature=None):
    outcome = {"model1": 0, "model2": 0, "draw": 0}
    for _ in range(games_per_side):
        b1 = battle(model1, model2, temperature=temperature)
        w1 = b1[0].winner

        if w1 is None:
            outcome["draw"] += 1
        elif w1 == "True":
            outcome["model1"] += 1
        else:
            outcome["model2"] += 1
        b2 = battle(model2, model1, temperature=temperature)

        w2 = b2[0].winner
        if w2 is None:
            outcome["draw"] += 1
        elif w2 == "True":
            outcome["model2"] += 1
        else:
            outcome["model1"] += 1
    return outcome


def choose_best_model(models):
    # let all the models play againts each other and then decide on who's the overall best
    pass

In [ ]:
from tqdm import tqdm


def rank_model(models, games_per_pair=10, temperature=None):
    n = len(models)

    # initialise stats
    stats = {
        i: {"index": i, "points": 0.0, "wins": 0, "draws": 0, "losses": 0, "games": 0}
        for i in range(n)
    }
    pairwise = {}

    # round-robin
    for i in tqdm(range(n)):
        for j in range(i + 1, n):
            res = war(
                models[i],
                models[j],
                games_per_side=games_per_pair,
                temperature=temperature,
            )

            wins_i = int(res.get("model1", 0))
            wins_j = int(res.get("model2", 0))
            draws = int(res.get("draw", 0))
            total = int(res.get("total_games", wins_i + wins_j + draws))

            # update pairwise store
            pairwise[(i, j)] = {
                "wins_i": wins_i,
                "wins_j": wins_j,
                "draws": draws,
                "total_games": total,
            }

            # convert to points
            points_i = wins_i * 1.0 + draws * 0.0 - wins_j * 1.0
            points_j = -points_i

            # update global stats
            stats[i]["points"] += points_i
            stats[j]["points"] += points_j

            stats[i]["wins"] += wins_i
            stats[j]["wins"] += wins_j
            stats[i]["draws"] += draws
            stats[j]["draws"] += draws

            stats[i]["losses"] += wins_j
            stats[j]["losses"] += wins_i

            stats[i]["games"] += total
            stats[j]["games"] += total

    ranking = sorted(
        [stats[k] for k in stats], key=lambda x: (-x["points"], -x["wins"], x["index"])
    )
    return {"ranking": ranking, "pairwise": pairwise, "summary": stats}